<a href="https://colab.research.google.com/github/Bainer13/Data_BBVA_Team_1/blob/Rodri/Proyecto_Bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Armado del entorno de trabajo



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [ ]:
ls

sample_data/  spark-3.3.2-bin-hadoop3.tgz


In [ ]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [ ]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [ ]:
!readlink -f $(which java) | sed "s:bin/java::"


/usr/lib/jvm/java-11-openjdk-amd64/


In [ ]:
!echo $JAVA_HOME

In [ ]:
!pip install -q findspark

In [ ]:
!pip install pyspark 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=06674267a6a44a6c7e946ae9447fd01868fd65dd9e96ddac500621f45e0fd824
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
import pyspark.sql.functions as F

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import col

In [ ]:
spark = SparkSession.builder.getOrCreate()

#Extraccion de datos

In [ ]:
spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'CORRECTED')

In [ ]:
#Lectura de parquets
df_address = spark.read.parquet('/content/sample_data/address.snappy.parquet')
df_customer = spark.read.parquet('/content/sample_data/customer.snappy.parquet')
df_customer_basics = spark.read.parquet('/content/sample_data/customer_basics.snappy.parquet')
df_email = spark.read.parquet('/content/sample_data/email.snappy.parquet')
df_phones = spark.read.parquet('/content/sample_data/phones.snappy.parquet')

In [ ]:
#Join entre clientes y telefonos
df_customer_basics_phones = df_customer_basics.join(df_phones, ['customer_id'], how='right').select('customer_id','phone_id')
df_customer_basics_phones.sort(F.asc("customer_id")).show()

+-----------+------------+
|customer_id|    phone_id|
+-----------+------------+
|   00000007|304920118694|
|   00000039|142040288534|
|   00000044|150642392069|
|   00000381|579839963031|
|   00005527|414386216022|
|   00005527|386965082277|
|   00013438|867720276639|
|   00021991|345682952298|
|   00038236|478970404176|
|   00041884|035048875817|
|   00048126|415719030992|
|   00048225|963827126204|
|   00052103|537419593373|
|   00064339|  7371137147|
|   00071569|441427295031|
|   00072996|432161765719|
|   00076782|  2766715319|
|   00077558|049475117647|
|   00084439|  6954764655|
|   00084439|358522593064|
+-----------+------------+
only showing top 20 rows



In [ ]:
#Join entre clientes y email
df_customer_basics_email = df_customer_basics.join(df_email,['customer_id'], how='inner').select('customer_id', 'email_desc')
df_customer_basics_email.show()

+-----------+----------+
|customer_id|email_desc|
+-----------+----------+
+-----------+----------+



In [ ]:
#join entre clientes y direccion
df_customer_basics_address = df_customer_basics.join(df_address,['customer_id']).select('customer_id', 'street_name','address_outdoor_id')
df_customer_basics_address.show()

+-----------+-----------+------------------+
|customer_id|street_name|address_outdoor_id|
+-----------+-----------+------------------+
+-----------+-----------+------------------+



In [ ]:
#Filtrado y reduccion de datos --> PHONES
w = Window.partitionBy(df_phones['customer_id']).orderBy("last_change_date")

In [ ]:
df_phones = df_phones.withColumn("row_num", row_number().over(w))

In [ ]:
df_phones = df_phones.filter(col("row_num") <= 3)

In [ ]:
df_phones = df_phones.select('customer_id', 'prefix_phone_id', 'phone_area_id', 'cellphone_prefix_id', 'phone_exchange_id', 'phone_line_id', 'phone_id').show(10)


+-----------+---------------+-------------+-------------------+-----------------+-------------+------------+
|customer_id|prefix_phone_id|phone_area_id|cellphone_prefix_id|phone_exchange_id|phone_line_id|    phone_id|
+-----------+---------------+-------------+-------------------+-----------------+-------------+------------+
|   00000007|             54|          266|                 15|              469|         7946|304920118694|
|   00000381|             54|          381|                 15|              590|         9885|579839963031|
|   00041884|             54|           11|                 15|             6157|         7947|035048875817|
|   00048225|             54|           11|                 15|             6518|         7983|963827126204|
|   00052103|             54|           11|                 15|             6995|         1912|537419593373|
|   00064339|             54|         2966|                   |               42|         5661|  7371137147|
|   00071569|      

In [ ]:
#Filtrado y reduccion de datos --> ADDRESS
w = Window.partitionBy(df_address['customer_id']).orderBy("last_change_date")

In [ ]:
df_address = df_address.withColumn("row_num", row_number().over(w))

In [ ]:
df_address = df_address.filter(col("row_num") <= 3)

In [ ]:
df_address = df_address.select('customer_id','street_name', 'address_outdoor_id', 'address_indoor_id', 'indoor_number', 'address_district_name', 'address_town_name', 'zipcode_id').show(10)

+--------------------+------------------+-----------------+-------------+---------------------+--------------------+----------+
|         street_name|address_outdoor_id|address_indoor_id|indoor_number|address_district_name|   address_town_name|zipcode_id|
+--------------------+------------------+-----------------+-------------+---------------------+--------------------+----------+
|AV PRES BARTOLOME...|           0001500|                 |             |                  ...|CRUCESITA        ...|     01870|
|CNEL AREVALO     ...|           0002391|                 |             |                  ...|CIUDAD AUTONOMA B...|     01425|
|VENEZUELA 538    ...|           23     |         1       |      CAPITAL| MONSERRAT        ...|                 ...|     11111|
|AV FELIX DE OLAZA...|           0005392|         1       |      C      |                  ...|CIUDAD AUTONOMA B...|     01431|
|HEROES DE MALVI  ...|           0001944|         00      |             |                  ...|LANUS    

In [ ]:
#Filtrado y reduccion de datos --> EMAILS
w = Window.partitionBy(df_email['customer_id']).orderBy("last_change_date")

In [ ]:
df_email = df_email.withColumn("row_num", row_number().over(w))

In [ ]:
df_email = df_email.filter(col("row_num") <= 3)

In [ ]:
df_email = df_email.select('customer_id', 'email_desc').show(10)

+-----------+--------------------+
|customer_id|          email_desc|
+-----------+--------------------+
|   00000000|LIBRERIAURUGUAY@G...|
|   00000000|ALVAROJOSE.DAGAND...|
|   00027555|ALEFRID666@YAHOO....|
|   00027568|   SCATIVA@GMAIL.COM|
|   00027595|        NO@GMAIL.COM|
|   00027609|lindocampo@yahoo.com|
|   00027726|MORELPUB@FIBERTEL...|
|   00027766|HAYDINEITOR@GMAIL...|
|   00027780|LEMOS-LILIANA@HOT...|
|   00027781|hgrinschpun@edeno...|
+-----------+--------------------+
only showing top 10 rows



In [ ]:
df_email.show(10) #Bloque para visualizar los datasets

+-----------+---------+-------------------+--------------+----------+------------------+--------------------+-----------------+--------------+-------------------+----------------+--------------------------+--------------+----------------+-------------------+--------------------+-----------------------+---------------------+--------------------------+-------------------+----------------+
|customer_id|role_type|address_sequence_id|residence_type|email_type|primary_email_type|          email_desc|email_domain_type|encripted_type|field_length_number|   comments_desc|customer_email_status_type|email_app_type|register_user_id|last_change_user_id|last_change_hms_date|last_change_terminal_id|operational_load_date|customer_email_status_date|registry_entry_date|last_change_date|
+-----------+---------+-------------------+--------------+----------+------------------+--------------------+-----------------+--------------+-------------------+----------------+--------------------------+--------------